In [1]:
import sys
import os
import requests
import json
import csv
import decimal
import copy

#sys.path.append("/smartcode/util")
import util
from importlib import reload
reload(util)

sys.version

'3.8.5 (default, May 27 2021, 13:30:53) \n[GCC 9.3.0]'

In [2]:
!pwd

/smartenv/setup/challenges


In [3]:
SMARTENV_PATH = "/smartenv/"
GEN_CONFIG_PATH = "/smartenv/setup/generated_config/"
GLOBAL_INFO_PATH = GEN_CONFIG_PATH + "/global_info/"
GENESIS_JSON = "/smartenv/setup/environment/genesis_config/go-ethereum/berlin/genesis.json"

PARTICIPANTS_PATH = "/smartenv/participants/"
PARTICIPANTS_CSV = PARTICIPANTS_PATH + "participants.csv"
PP_CSV = PARTICIPANTS_CSV

SCRIPTS_PATH = "/smartenv/util/scripts/"
CHALLENGES_PATH = "/smartenv/challenges/"
CHL_NOTAWALLET_PATH = "/smartenv/challenges/notawallet/"

In [4]:
SRVIP = "172.18.0.8"
SRVPORT = "8545"
SRVCHAINID = "20221011" # CHANGE THIS IS UPDATED!
SRVNETVER = "25519"

In [5]:
SEED_VALUE = 16*10**18
SEED_VALUE_NOTAWALLET = 10*10**18
SEED_VALUE_BADPARITY = 30*10**18
SEED_VALUE_DAODOWN = 30*10**18
SEED_VALUE_FAILDICE = 30*10**18

# Challenge setup geth

## Start environment: smartenv-geth & smartenv-web3py 

We will run the server/geth node as well as the smar>tenv-web3py client docker containers on the same host machine. 

```bash
$ make run-smartenv-web3py # the container running this notebook
$ make run-smartenv-geth # the geth client this notebook connectes to
```


## Connect and check chain state

Note that for *PoA* nodes the middleware needs to be added.
Therefore, set the `poa` flag to true when connecting. 

In [8]:
#w3 = web3.Web3(web3.Web3.HTTPProvider("http://172.18.0.1:8545"))
#w3=util.connect(host="172.18.0.5",port="8544",poa=True) 
w3=util.connect(host=SRVIP,port=SRVPORT,poa=True)
# check if connection was successful
assert w3.isConnected()

In [9]:
w3.api #web3 version used

'5.20.0'

In [10]:
w3.clientVersion # remote client version to which w3 is connected 

'Geth/bob/v1.10.3-stable-991384a7/linux-amd64/go1.15.4'

In [11]:
print(w3.net.version) # in the console: admin.nodeInfo
assert w3.net.version == SRVNETVER

25519


In [12]:
print(w3.eth.chainId) # in the console: admin.nodeInfo
assert w3.eth.chainId == int(SRVCHAINID)

20221011


In [13]:
w3.eth.blockNumber

609

In [14]:
w3.eth.getBlock(w3.eth.blockNumber)["gasLimit"]

8249765

In [15]:
w3.eth.mining

False

In [17]:
w3.eth.syncing

False

### Networ check

In [18]:
w3.net.peerCount

1

In [19]:
w3.geth.admin.peers()

[AttributeDict({'enode': 'enode://ed29c7ec06fdc64489d66ec1e714235b22ced1e5a81de01470f6da0e3dcfdaa0872af1a259536df0641b653072bfaa633c5d6308b578ed7b027f9d5b691c822b@131.130.126.71:30303',
  'id': '60d5df66f8e7b95eaa31021a62cdce511522cfd49b27cdf3011e6356bfcbe4e1',
  'name': 'Geth/alice/v1.10.3-stable-991384a7/linux-amd64/go1.15.4',
  'caps': ['eth/65', 'eth/66', 'snap/1'],
  'network': AttributeDict({'localAddress': '172.18.0.8:50550',
   'remoteAddress': '131.130.126.71:30303',
   'inbound': False,
   'trusted': False,
   'static': False}),
  'protocols': AttributeDict({'eth': AttributeDict({'version': 66,
    'difficulty': 1225,
    'head': '0xab181f599965036e95888b1fd303cde799c5f374adb8614f8783a1877bf9d1bb'}),
   'snap': AttributeDict({'version': 1})})})]

In [20]:
w3.net.listening

True

In [21]:
w3.eth.accounts

['0xb1Cef50494b3C82Aa199DCedD212D333C9265697',
 '0x22f9752D7933D89bE858e9a6Fbd0f6CAE2aDAAE0',
 '0x1DE8Ea7FD0fE6E656964a68665DAe4Cc1eB630F1']

### Export (correct) enode info for server (docker container)

Delete the old generated config dir beforehand:

In [45]:
GEN_CONFIG_PATH

'/smartenv/setup/generated_config/'

In [46]:
!mkdir -p {GEN_CONFIG_PATH} 

In [47]:
poa_node_con = False # if connected directly to the poa node switch to True

enode_file = GLOBAL_INFO_PATH + "/enodes"
# only needed if running directly on PoA node server:
#invalid_ip_port = "172.18.0.5:30303"
invalid_ip_port = "127.0.0.1:30303"
valid_ip_port = "128.131.169.35:30303"

if poa_node_con:
    if w3.clientVersion.startswith('Parity'):
        enode = w3.parity.enode()
    elif w3.clientVersion.startswith('Geth'):
        enode = w3.geth.admin.nodeInfo()['enode']
    enode = enode.replace(invalid_ip_port,valid_ip_port)
else:
    enode = w3.geth.admin.peers()[0]["enode"]
enode

'enode://ed29c7ec06fdc64489d66ec1e714235b22ced1e5a81de01470f6da0e3dcfdaa0872af1a259536df0641b653072bfaa633c5d6308b578ed7b027f9d5b691c822b@131.130.126.71:30303'

In [48]:
# Write enode to file:
!mkdir -p $GLOBAL_INFO_PATH
!echo $enode > $enode_file

In [26]:
# block 0 ist the genesis block
genesis_block = w3.eth.getBlock(0)
genesis_block

AttributeDict({'difficulty': 1,
 'proofOfAuthorityData': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000d6abfd59de4b5c832f74d8f0ab9fd844e0ad02550000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'gasLimit': 15000000,
 'gasUsed': 0,
 'hash': HexBytes('0x4e27219a6b230d7b16326006da6ae29a33d350d35070d9f0eefa58d6a06e7c4b'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'miner': '0x00000000000

Copy password file over for generated accounts:

In [49]:
!cp {SMARTENV_PATH}/passwordfile {GLOBAL_INFO_PATH}

Copy `genesis.json` over for generated config:

In [50]:
!cp {GENESIS_JSON} {GLOBAL_INFO_PATH}

## Import students

Dowload and prepare studentd csv file:
```
$ cat participants_students_original.csv | awk -F, '{print $1","$2","$4","$3}' > participants_students.csv
$ cat participants_template.csv participants_students.csv participants_placeholder.csv > participants.csv
```

In [29]:
print(PARTICIPANTS_PATH)
print(PARTICIPANTS_CSV)
print(PP_CSV)

/smartenv/participants/
/smartenv/participants/participants.csv
/smartenv/participants/participants.csv


In [30]:
!ls -la {PARTICIPANTS_CSV}

lrwxrwxrwx 1 smartenv smartenv 31 Oct 11 14:54 /smartenv/participants/participants.csv -> participants_052011_1_2022W.csv


In [31]:
!head -n2 $PARTICIPANTS_CSV

Aljosha,Judmayer,ajudmayer@sba-research.org,1
Nicholas,Stifter,nstifter@sba-research.org,2


In [32]:
!file $PARTICIPANTS_CSV # get file format/encoding

/smartenv/participants/participants.csv: symbolic link to participants_052011_1_2022W.csv


In [33]:
!wc -l $PARTICIPANTS_CSV

16 /smartenv/participants/participants.csv


In [34]:
wc_out = !wc -l $PARTICIPANTS_CSV # check number of stutend and compare later 
pp_count_wc = int(wc_out[0].split(" ")[0])
pp_count_wc

16

In [35]:
pp_list = list() # list of participants
pp_count = 0 # number of participants

in_fieldnames = ['firstname', 'lastname','email','ppid'] # ppid could be matr.

In [36]:
def read_students(path,pp_list):
    with open(path,'r',encoding='utf-8') as f_input:
    #with open(path,'r',encoding='utf-16') as f_input:
    #with open(path,'r',encoding='latin-1') as f_mail:
        #sreader = csv.DictReader(f_input, fieldnames=in_fieldnames, delimiter='\t',quotechar='\"')
        sreader = csv.DictReader(f_input, fieldnames=in_fieldnames, delimiter=',',quotechar='\"')
        for i,row in enumerate(sreader):
            #print(i,row)
            pp_dict = dict()
            for field in in_fieldnames:
                    pp_dict[field]=row[field]
            pp_list.append(pp_dict)
            #print(students_list)
    pp_count = i+1
    return pp_count

In [37]:
read_students(PP_CSV,pp_list)

16

In [38]:
pp_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1'}

In [39]:
pp_list[-1]

{'firstname': 'placeholder',
 'lastname': 'placeholder',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '16'}

In [40]:
# pp_list # long output

In [41]:
pp_count = len(pp_list)
assert pp_count == pp_count_wc
pp_count

16

## Create keys

In [42]:
SCRIPTS_PATH

'/smartenv/util/scripts/'

In [43]:
!cat {SCRIPTS_PATH}create_accounts.sh 

#!/bin/bash
#
# Run from root of repository

set -e

if [ -z "${1}" ] && [ -z "${PARTICIPANTS+x}" ];
then
    echo "PARTICIPANTS = No number given e.g., '42'"
    exit 2
elif [ ! -z "${1}" ];
then
    export PARTICIPANTS="${1}"
fi
echo "PARTICIPANTS = ${PARTICIPANTS}"

if [ -z "${DATADIR+x}" ];
then
    echo "DATADIR environment variable missing"
	echo "No Datadir given e.g., './datadir/alice/'"
	exit 2
fi
echo "DATADIR = ${DATADIR}"

if [ -z "${TARGETDIR+x}" ];
then
        echo "TARGETDIR environment variable missing"
	echo "No Datadir given e.g., './setup/generated_config/keystore'"
	exit 2
fi
echo "TARGETDIR = ${TARGETDIR}"

mkdir -p ${TARGETDIR}

#for i in {0..90}};
for i in $(seq 1 ${PARTICIPANTS});
do 
	echo $i
	bash ./util/scripts/geth_account.sh new
done

cp -r "${DATADIR}/keystore" ${TARGETDIR}


In [44]:
!cat {SCRIPTS_PATH}geth_account.sh

#!/bin/bash
#
#
# Run with docker from root of repository:
# $ PWFILE=./passwordfile DATADIR=./datadir/accountmgt/ bash geth_account.sh new

set -e

if [ -z "${PWFILE+x}" ];
then
	echo "PWFILE environment variable missing"
	echo "No Password file given using default './passwordfile'"
	export PWFILE='./passwordfile'
fi
echo "PWFILE = ${PWFILE}"

if [ -f "${PWFILE}" ];
then
	echo "using stored password ... "	
else
	#pwgen -s -B 32 1 > ${PWFILE}
	echo "ERROR: No passwordfile found!"
	exit 1
fi

if [ -z "${DATADIR+x}" ];
then
        echo "DATADIR environment variable missing"
	echo "No Datadir given e.g., './datadir/bob'"
	exit 2
fi
echo "DATADIR = ${DATADIR}"

mkdir -p ${DATADIR}

# run container without network
# just to generate and manage keys
docker run \
	--mount type=bind,source=$(pwd),target=/smartenv \
	-it smartenv-geth:latest \
		geth \
		--datadir "${DATADIR}" \
		--nodiscover \
		--maxpeers 0 \
		--ipcdisable \
		--verbosity 6  \
		--password "${PWFILE}" \
		account "$1" "$2"

Run the following command from the root folder of the repository: 
(Take care that there are not more keys under the keystore in DATADIR or the TARGETDIR)

## Assign addresses to users (testaccounts & students)

In [51]:
#### CHANGE ####
 
# path where the raw ethereuem keys are located:
#source_path="./test/keystore/" 
source_path= GEN_CONFIG_PATH + "/keystore"

# Either run with test users or with all students:
#users_list = copy.deepcopy(testers_list)
users_list = copy.deepcopy(pp_list)

#users_list = copy.deepcopy(testers_list)
#users_list.extend(copy.deepcopy(students_list)

#### CHANGE END ####

In [52]:
# print some user as test before assignment
print(len(users_list))
users_list[0]

16


{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1'}

In [53]:
!ls {GEN_CONFIG_PATH}

global_info  keystore


In [54]:
!cd $source_path && pwd && ls | head -n1

/smartenv/setup/generated_config/keystore
UTC--2021-11-05T10-35-42.522469976Z--b4ebf9c576cac2b3c3caf17860bb70a6b02201f3


In [55]:
#!cd $source_path && ls ./ | head -n1 | xargs -Irpl cat rpl

In [56]:
def read_keypairs(path):
    """read keypairs from keystore
    """
    _keypairs = list()
    # read out keystore folder 
    for root,dirs,files in os.walk(path):
        #print("root = {}".format(root))
        #print("dirs = {}".format(dirs))
        for f in files:
            with open(root + "/" + f,"r",encoding="utf-8") as f_input:
                data = json.loads(f_input.read()) # load all key pair data from file
                data["filename"] = f # add file name to data for later
                _keypairs.append(data)
    return _keypairs   

In [57]:
def assign_addresses(keystore_path,users,attach_path="/smartenv/setup/generated_config/attach"):
    """ map keypairs to users and add info to user dict
    This also creates the attach directory. 
    """
    keypairs = read_keypairs(keystore_path)
    assert len(keypairs) >= len(users)
    
    for u in users:
        kp = keypairs.pop()
        u["account"] = kp["address"]
        #u["keypair"] = kp
        #u["key"] = kp["crypto"]["ciphertext"]
        u["filename"] = kp["filename"]
        
        arg_dest = attach_path + "/" + u["account"]
        !mkdir -p $arg_dest
        arg_src = keystore_path + "/" + kp["filename"]
        !cp $arg_src $arg_dest
        u["attach"] = "/tmp/attach/" + u["account"] + ".tar.gz"
        print(".",end=".")
    return users

In [58]:
attach_path= GEN_CONFIG_PATH + "/attach"
#!rm -rf attach_path
assign_addresses(source_path,users_list,attach_path)

# print some user as test after assignment
users_list[0]

................................

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '6a78cb4b4514091c91e81efcb73e132f0e050345',
 'filename': 'UTC--2022-10-11T13-21-16.771622196Z--6a78cb4b4514091c91e81efcb73e132f0e050345',
 'attach': '/tmp/attach/6a78cb4b4514091c91e81efcb73e132f0e050345.tar.gz'}

In [59]:
def getBalance(address):
    address = w3.toChecksumAddress(address)
    return w3.fromWei(w3.eth.getBalance(address),'ether')

In [60]:
getBalance(users_list[0]["account"])

0

In [61]:
w3.eth.accounts

['0xb1Cef50494b3C82Aa199DCedD212D333C9265697',
 '0x22f9752D7933D89bE858e9a6Fbd0f6CAE2aDAAE0',
 '0x1DE8Ea7FD0fE6E656964a68665DAe4Cc1eB630F1']

In [62]:
for a in w3.eth.accounts:
    print(a,": ",getBalance(a))

0xb1Cef50494b3C82Aa199DCedD212D333C9265697 :  1000000
0x22f9752D7933D89bE858e9a6Fbd0f6CAE2aDAAE0 :  1000000
0x1DE8Ea7FD0fE6E656964a68665DAe4Cc1eB630F1 :  1000000


In [63]:
w3.eth.defaultAccount = w3.eth.accounts[0]

In [64]:
w3.eth.defaultAccount

'0xb1Cef50494b3C82Aa199DCedD212D333C9265697'

In [65]:
def send_accounts_money(users,amount):
    for u in users:
        account = w3.toChecksumAddress(u["account"])
        u["seedbalance"] = amount
        w3.eth.sendTransaction({"from":w3.eth.defaultAccount,"to":account,"value":amount})
    return

In [66]:
w3.geth.personal.unlockAccount(w3.eth.accounts[0],"notforproductiveuse",0)

True

In [67]:
send_accounts_money(users_list,SEED_VALUE)

In [68]:
!sleep 16
getBalance(users_list[0]["account"])

Decimal('16')

In [69]:
getBalance(users_list[-1]["account"])

Decimal('16')

In [70]:
def check_accounts_money(users,predicate=(lambda a, b: a == b)):
    for u in users:
        u_balance = w3.eth.getBalance(w3.toChecksumAddress(u["account"]))
        u["balance"] = u_balance
        u_seedbalance = u["seedbalance"]
        if predicate(u_balance,u_seedbalance) == False:
            print("Predicate violation for:\n",u)
            assert False,"Predicate violation"

In [71]:
check_accounts_money(users_list)

In [72]:
check_accounts_money(users_list,predicate=(lambda a, b: a >= b))
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '6a78cb4b4514091c91e81efcb73e132f0e050345',
 'filename': 'UTC--2022-10-11T13-21-16.771622196Z--6a78cb4b4514091c91e81efcb73e132f0e050345',
 'attach': '/tmp/attach/6a78cb4b4514091c91e81efcb73e132f0e050345.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000}

### Test manual account handling

### Deployment functions

In [73]:
sys.path.append( SMARTENV_PATH + "/util")
import util
from importlib import reload
reload(util)

<module 'util' from '/smartenv/setup/challenges/util/__init__.py'>

In [74]:
def forall_compile_and_deploy(users,
                              path,
                              contract_name,
                              value,
                              force_argument=None,
                              argument2=None,
                              compiler="solc"):
    print("Started for {} users:".format(len(users)))
    for u in users:
        if force_argument is not None:
            print("*",end="")
            #print("Force argument: {}".format(force_argument))
            u_addr = force_argument
        else:
            u_addr = w3.toChecksumAddress(u["account"])
            
        if argument2 is not None and argument2 is True:
            print("~",end="")
            #print("Double sha3 argument2: {}".format(argument2))
            tx_hash = util.compile_and_deploy_contract(path,
                                                   patch_api=False,
                                                   concise=True,
                                                   argument=u_addr,
                                                   argument2=w3.toInt(hexstr=w3.sha3(text=w3.sha3(text=u_addr).hex()).hex()),
                                                   wait=False,
                                                   value=value,
                                                   compiler=compiler)
        elif argument2 is not None and isinstance(argument2,type(str())):
            print("+",end="")
            #print("Custom argument2: {}".format(argument2))
            tx_hash = util.compile_and_deploy_contract(path,
                                                   patch_api=False,
                                                   concise=True,
                                                   argument=u_addr,
                                                   argument2=argument2,
                                                   wait=False,
                                                   value=value,
                                                   compiler=compiler)
        else:
            print(".",end="")
            tx_hash = util.compile_and_deploy_contract(path,
                                                   patch_api=False,
                                                   concise=True,
                                                   argument=u_addr,
                                                   wait=False,
                                                   value=value,
                                                   compiler=compiler)
        # returns HexBytes
        # import hexbytes
        # help(hexbytes.main)
        u[contract_name + "_tx"] = tx_hash.hex()
        u[contract_name + "_seedvalue"] = value
        
    for u in users:
        tx_receipt = w3.eth.waitForTransactionReceipt(u[contract_name + "_tx"])
        u[contract_name + "_addr"] = tx_receipt['contractAddress']
        print("-",end="")
        
    return users

In [75]:
def forall_check_contract(users,
                          c_path,
                          c_name,
                          predicate=(lambda a, b: a == b),
                          compiler="solc"):
    # Check for all users if the contract is initialized to the respective user
    # and that the balance is the seed value
    print("Started for {} users:".format(len(users)))
    for u in users:
        c_abi = util.compile_contract_with_libs(compiler_path=compiler,src_path=c_path)["abi"]
        u_c_addr = u[c_name + "_addr"]
        u_c_instance = util.get_contract_instance(u_c_addr,c_abi,concise=False)
    
        # check owner
        #print(u_c_instance.functions.getStudent().call())
        #print(w3.toChecksumAddress(u["account"]))
        assert u_c_instance.functions.getStudent().call() == w3.toChecksumAddress(u["account"])   
        # check balance
        u_c_seedvalue = u[c_name + "_seedvalue"]
        u_c_balance = w3.eth.getBalance(u_c_addr)   
        u[c_name + "_balance"] = u_c_balance
        if predicate(u_c_balance,u_c_seedvalue) == False:
            print("Predicate violation for:\n",u)
        print(".",end="")
    return users

In [76]:
def copy_challenge_metadata(users,challenge_path,challenge_name,attach_path="./attach"):
    # copy challenge metadata to individual user folders helper function
    for d in os.listdir(attach_path):
        #print(d)
        if os.path.isdir(os.path.join(attach_path, d)):
            #print(d)
            arg_dest_dir = attach_path + "/" + d + "/" + challenge_name
            print(arg_dest_dir)
            !mkdir -p $arg_dest_dir
            !cp -r $challenge_path $arg_dest_dir

# Challange: notawallet 

In [77]:
CHL_NOTAWALLET_PATH

'/smartenv/challenges/notawallet/'

In [78]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '6a78cb4b4514091c91e81efcb73e132f0e050345',
 'filename': 'UTC--2022-10-11T13-21-16.771622196Z--6a78cb4b4514091c91e81efcb73e132f0e050345',
 'attach': '/tmp/attach/6a78cb4b4514091c91e81efcb73e132f0e050345.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000}

In [79]:
# users_list[-1]
users_list[1]

{'firstname': 'Nicholas',
 'lastname': 'Stifter',
 'email': 'nstifter@sba-research.org',
 'ppid': '2',
 'account': '9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'filename': 'UTC--2021-11-05T10-36-24.055208650Z--9ddbee8f1bb05273118d8e802797e5fb93fc1bc0',
 'attach': '/tmp/attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000}

In [80]:
!solc --version

solc, the solidity compiler commandline interface
Version: 0.8.4+commit.c7e474f2.Linux.g++


In [81]:
!solc --overwrite -o "{CHL_NOTAWALLET_PATH}/publish/" --abi --bin "{CHL_NOTAWALLET_PATH}/publish/NotAWallet.sol"

Compiler run successful. Artifact(s) can be found in directory /smartenv/challenges/notawallet//publish/.


In [82]:
## Connections refresh maybe needed
w3=util.connect(host=SRVIP,port=SRVPORT,poa=True) 
assert w3.isConnected()

In [83]:
path= CHL_NOTAWALLET_PATH + "/publish/NotAWallet.sol"
challange_name="notawallet"
seedvalue=10*10**18

forall_compile_and_deploy(users_list,path,challange_name,seedvalue)
users_list[0]

Started for 16 users:
................----------------

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '6a78cb4b4514091c91e81efcb73e132f0e050345',
 'filename': 'UTC--2022-10-11T13-21-16.771622196Z--6a78cb4b4514091c91e81efcb73e132f0e050345',
 'attach': '/tmp/attach/6a78cb4b4514091c91e81efcb73e132f0e050345.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xf85c1a0d3c5349d234b1392a32ea23b749c10d8f9161f8aca8fa8cf4dc6be10b',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xd1cEE4d9114ba7c29489bfd65487Dd04B39cd386'}

In [84]:
!solc --version

solc, the solidity compiler commandline interface
Version: 0.8.4+commit.c7e474f2.Linux.g++


In [85]:
c_abi = util.compile_contract_with_libs(compiler_path="solc",src_path=CHL_NOTAWALLET_PATH + "/publish/NotAWallet.sol")["abi"]
c_abi

[{'inputs': [{'internalType': 'address',
    'name': '_student',
    'type': 'address'}],
  'stateMutability': 'payable',
  'type': 'constructor'},
 {'inputs': [{'internalType': 'address',
    'name': 'newowner',
    'type': 'address'}],
  'name': 'addOwner',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'inputs': [],
  'name': 'deposit',
  'outputs': [],
  'stateMutability': 'payable',
  'type': 'function'},
 {'inputs': [],
  'name': 'getStudent',
  'outputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [{'internalType': 'address',
    'name': 'testowner',
    'type': 'address'}],
  'name': 'isOwner',
  'outputs': [{'internalType': 'bool', 'name': '', 'type': 'bool'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [],
  'name': 'owner',
  'outputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
  'stateMutability': 'view',
  'type': 'functi

In [86]:
w3.toChecksumAddress(users_list[0]["account"])

'0x6a78cB4B4514091c91E81EfCb73E132F0e050345'

In [87]:
notawallet_addr = users_list[0]["notawallet_addr"]
notawallet_addr

'0xd1cEE4d9114ba7c29489bfd65487Dd04B39cd386'

In [88]:
u_c_instance = util.get_contract_instance(notawallet_addr,c_abi,concise=False)

In [89]:
assert u_c_instance.functions.getStudent().call() == w3.toChecksumAddress(users_list[0]["account"])
u_c_instance.functions.getStudent().call()

'0x6a78cB4B4514091c91E81EfCb73E132F0e050345'

In [90]:
path=CHL_NOTAWALLET_PATH + "/publish/NotAWallet.sol"
challange_name="notawallet"
forall_check_contract(users_list,path,challange_name)
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a < b))
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a == 10**18))
users_list[0]

Started for 16 users:
................

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '6a78cb4b4514091c91e81efcb73e132f0e050345',
 'filename': 'UTC--2022-10-11T13-21-16.771622196Z--6a78cb4b4514091c91e81efcb73e132f0e050345',
 'attach': '/tmp/attach/6a78cb4b4514091c91e81efcb73e132f0e050345.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xf85c1a0d3c5349d234b1392a32ea23b749c10d8f9161f8aca8fa8cf4dc6be10b',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xd1cEE4d9114ba7c29489bfd65487Dd04B39cd386',
 'notawallet_balance': 10000000000000000000}

In [91]:
# copy challenge metadata
challenge_path= CHL_NOTAWALLET_PATH + "/publish/*"
attach_path=GEN_CONFIG_PATH + "/attach"
challenge_name="notawallet"
copy_challenge_metadata(users_list,challenge_path,challenge_name,attach_path)

/smartenv/setup/generated_config//attach/df989609ed61dea253bb838927c66818142384a7/notawallet
/smartenv/setup/generated_config//attach/ec783a2c453025391c4e70875c8cf4f34fa2ab1b/notawallet
/smartenv/setup/generated_config//attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0/notawallet
/smartenv/setup/generated_config//attach/dc870b8a86f3d2201fbdbad4e3995154c416e889/notawallet
/smartenv/setup/generated_config//attach/6a78cb4b4514091c91e81efcb73e132f0e050345/notawallet
/smartenv/setup/generated_config//attach/f15e7785122b2725c5438d79b4ea302c48961f28/notawallet
/smartenv/setup/generated_config//attach/30b0fe2ed4e20e16944d9f032cfb0e9067eb5479/notawallet
/smartenv/setup/generated_config//attach/0a25d85f850060cfcd6bbbf95825ed0e8ef24e6b/notawallet
/smartenv/setup/generated_config//attach/b69743e47ec0dcbc45ea0260c96469d72e56fd0a/notawallet
/smartenv/setup/generated_config//attach/c0af8d74d633cca61ddd57f122aef2d03ce20757/notawallet
/smartenv/setup/generated_config//attach/5e5d6f287e50fd9308634c2ac892c

# Challenge: badparity

In [92]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '6a78cb4b4514091c91e81efcb73e132f0e050345',
 'filename': 'UTC--2022-10-11T13-21-16.771622196Z--6a78cb4b4514091c91e81efcb73e132f0e050345',
 'attach': '/tmp/attach/6a78cb4b4514091c91e81efcb73e132f0e050345.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xf85c1a0d3c5349d234b1392a32ea23b749c10d8f9161f8aca8fa8cf4dc6be10b',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xd1cEE4d9114ba7c29489bfd65487Dd04B39cd386',
 'notawallet_balance': 10000000000000000000}

In [93]:
!cat "{CHALLENGES_PATH}/badparity/publish/Wallet.sol"

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.4;

contract Wallet {
    address walletLibrary;
    address payable owner;
    address payable student;

    event LogValue(uint256 exitcode,uint256 amount);

    // constructor, called once when this contract is created 
    constructor(address payable _student, address lib) payable {
        student = _student;  
        walletLibrary = lib; // hardcode lib address at deploy time
        // init the owner with the respective lib contract
        walletLibrary.delegatecall(abi.encodeWithSignature("initWallet(address)", msg.sender));
    }

    function getOwner() public view returns (address payable) {
        return owner;
    } 

    function getWalletLibrary() public view returns (address) {
        return walletLibrary;
    }

    function withdraw(uint256 amount) public returns (bool) {
        (bool success, bytes memory data) = walletLibrary.delegatecall(abi.encodeWithSignature("withdraw(uint256)", amount));
        if ( suc

In [94]:
!solc --overwrite -o "{CHALLENGES_PATH}/badparity/publish/" --abi --bin "{CHALLENGES_PATH}/badparity/publish/Wallet.sol"

 --> /smartenv/challenges//badparity/publish/Wallet.sol:4:1:
  |
4 | contract Wallet {
  | ^ (Relevant source part starts here and spans across multiple lines).
Note: The payable fallback function is defined here.
  --> /smartenv/challenges//badparity/publish/Wallet.sol:47:5:
   |
47 |     fallback() external payable {
   |     ^ (Relevant source part starts here and spans across multiple lines).

  --> /smartenv/challenges//badparity/publish/Wallet.sol:16:9:
   |
16 |         walletLibrary.delegatecall(abi.encodeWithSignature("initWallet(address)", msg.sender));
   |         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  --> /smartenv/challenges//badparity/publish/Wallet.sol:50:9:
   |
50 |         walletLibrary.delegatecall(msg.data);
   |         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  --> /smartenv/challenges//badparity/publish/Wallet.sol:28:24:
   |
28 |         (bool success, bytes memory data) = walletLibrary.delegatecall(abi.encodeWith

In [95]:
!solc --overwrite -o "{CHALLENGES_PATH}/badparity/publish/" --abi --bin "{CHALLENGES_PATH}/badparity/publish/WalletLibrary.sol"

 --> /smartenv/challenges//badparity/publish/WalletLibrary.sol:4:1:
  |
4 | contract WalletLibrary {
  | ^ (Relevant source part starts here and spans across multiple lines).
Note: The payable fallback function is defined here.
  --> /smartenv/challenges//badparity/publish/WalletLibrary.sol:36:6:
   |
36 |      fallback() external payable {
   |      ^ (Relevant source part starts here and spans across multiple lines).



In [96]:
# compile and deploy lib once for all users
path=CHALLENGES_PATH + "/badparity/publish/WalletLibrary.sol"

lib = util.compile_and_deploy_contract(path,
                                 patch_api=False,
                                 concise=False)
lib.address

'0x856D84841B7d77e9f273CC5Fc4A55df10B1AF75a'

In [97]:
path=CHALLENGES_PATH + "/badparity/publish/Wallet.sol"
challange_name="badparity"
seedvalue=SEED_VALUE_BADPARITY

forall_compile_and_deploy(users_list,path,challange_name,seedvalue,argument2=lib.address)

for u in users_list:
    u[challange_name + "_lib"] = lib.address

users_list[0]

Started for 16 users:
++++++++++++++++----------------

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '6a78cb4b4514091c91e81efcb73e132f0e050345',
 'filename': 'UTC--2022-10-11T13-21-16.771622196Z--6a78cb4b4514091c91e81efcb73e132f0e050345',
 'attach': '/tmp/attach/6a78cb4b4514091c91e81efcb73e132f0e050345.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xf85c1a0d3c5349d234b1392a32ea23b749c10d8f9161f8aca8fa8cf4dc6be10b',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xd1cEE4d9114ba7c29489bfd65487Dd04B39cd386',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x3a3aa71994d7f4f7fb95cf576d216b57a20e2d6f26077f848fb3c9dc6100e4c2',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0x5d37ef7d151A7c2bB7F491CaB359aB618c8a08B0',
 'badparity_lib': '0x856D84841B7d77e9f273CC5Fc4A55df10B1AF75a'}

In [98]:
# spot check of one contract
c_abi = util.compile_contract_with_libs("solc",path)["abi"]
u_c_addr = users_list[0][challange_name + "_addr"]
u_c_instance = util.get_contract_instance(u_c_addr,c_abi,concise=True)

assert users_list[0][challange_name + "_lib"] == lib.address
    
# check lib
assert str(u_c_instance.getWalletLibrary()).lower() == str(lib.address).lower()
assert str(u_c_instance.getOwner()).lower() == str(w3.eth.accounts[0]).lower()

# check balance
assert w3.eth.getBalance(u_c_addr) == SEED_VALUE_BADPARITY

In [99]:
u_c_instance.getOwner()

'0xb1Cef50494b3C82Aa199DCedD212D333C9265697'

In [100]:
u_c_instance.getStudent()

'0x6a78cB4B4514091c91E81EfCb73E132F0e050345'

In [101]:
w3.parity.personal.listAccounts()

['0xb1Cef50494b3C82Aa199DCedD212D333C9265697',
 '0x22f9752D7933D89bE858e9a6Fbd0f6CAE2aDAAE0',
 '0x1DE8Ea7FD0fE6E656964a68665DAe4Cc1eB630F1']

In [102]:
# copy challenge metadata
challenge_path=CHALLENGES_PATH + "/badparity/publish/*"
attach_path=GEN_CONFIG_PATH + "/attach"
challenge_name="badparity"
copy_challenge_metadata(users_list,challenge_path,challenge_name,attach_path)

/smartenv/setup/generated_config//attach/df989609ed61dea253bb838927c66818142384a7/badparity
/smartenv/setup/generated_config//attach/ec783a2c453025391c4e70875c8cf4f34fa2ab1b/badparity
/smartenv/setup/generated_config//attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0/badparity
/smartenv/setup/generated_config//attach/dc870b8a86f3d2201fbdbad4e3995154c416e889/badparity
/smartenv/setup/generated_config//attach/6a78cb4b4514091c91e81efcb73e132f0e050345/badparity
/smartenv/setup/generated_config//attach/f15e7785122b2725c5438d79b4ea302c48961f28/badparity
/smartenv/setup/generated_config//attach/30b0fe2ed4e20e16944d9f032cfb0e9067eb5479/badparity
/smartenv/setup/generated_config//attach/0a25d85f850060cfcd6bbbf95825ed0e8ef24e6b/badparity
/smartenv/setup/generated_config//attach/b69743e47ec0dcbc45ea0260c96469d72e56fd0a/badparity
/smartenv/setup/generated_config//attach/c0af8d74d633cca61ddd57f122aef2d03ce20757/badparity
/smartenv/setup/generated_config//attach/5e5d6f287e50fd9308634c2ac892c6a6e02506c

# Challenge: Daodown

In [103]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '6a78cb4b4514091c91e81efcb73e132f0e050345',
 'filename': 'UTC--2022-10-11T13-21-16.771622196Z--6a78cb4b4514091c91e81efcb73e132f0e050345',
 'attach': '/tmp/attach/6a78cb4b4514091c91e81efcb73e132f0e050345.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xf85c1a0d3c5349d234b1392a32ea23b749c10d8f9161f8aca8fa8cf4dc6be10b',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xd1cEE4d9114ba7c29489bfd65487Dd04B39cd386',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x3a3aa71994d7f4f7fb95cf576d216b57a20e2d6f26077f848fb3c9dc6100e4c2',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0x5d37ef7d151A7c2bB7F491CaB359aB618c8a08B0',
 'badparity_lib': '0x856D84841B7d77e9f273CC5Fc4A55df10B1AF75a'}

In [104]:
# !solc --overwrite -o "{CHALLENGES_PATH}/daodown/publish/" --abi --bin "{CHALLENGES_PATH}/daodown/publish/EDao.sol"

In [105]:
!solc_7.4 --overwrite -o "{CHALLENGES_PATH}/daodown/publish/" --abi --bin "{CHALLENGES_PATH}/daodown/publish/EDao.sol"

Compiler run successful. Artifact(s) can be found in directory /smartenv/challenges//daodown/publish/.


In [106]:
path=CHALLENGES_PATH + "/daodown/publish/EDao.sol"
challange_name="daodown"
seedvalue=SEED_VALUE_DAODOWN

forall_compile_and_deploy(users_list,
                          path,
                          challange_name,
                          seedvalue,
                          compiler="solc_7.4")
users_list[0]

Started for 16 users:
................----------------

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '6a78cb4b4514091c91e81efcb73e132f0e050345',
 'filename': 'UTC--2022-10-11T13-21-16.771622196Z--6a78cb4b4514091c91e81efcb73e132f0e050345',
 'attach': '/tmp/attach/6a78cb4b4514091c91e81efcb73e132f0e050345.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xf85c1a0d3c5349d234b1392a32ea23b749c10d8f9161f8aca8fa8cf4dc6be10b',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xd1cEE4d9114ba7c29489bfd65487Dd04B39cd386',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x3a3aa71994d7f4f7fb95cf576d216b57a20e2d6f26077f848fb3c9dc6100e4c2',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0x5d37ef7d151A7c2bB7F491CaB359aB618c8a08B0',
 'badparity_lib': '0x856D84841B7d77e9f273CC5Fc4A55df10B1AF75a',
 'daodown_tx': '0x242de5bde50ebb3b1e4df1723b6f206296a2f7d8a7fe3ebfcc55fb7faf7eb48e',
 'daodo

In [107]:
w3.eth.blockNumber

663

In [108]:
w3.eth.getBlock(w3.eth.blockNumber)["gasUsed"]

703175

In [109]:
w3.eth.getBlock(w3.eth.blockNumber-3)["gasUsed"]

3515875

In [110]:
path=CHALLENGES_PATH + "/daodown/publish/EDao.sol"
challange_name="daodown"
forall_check_contract(users_list,
                      path,
                      challange_name,
                      compiler="solc_7.4")
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a < b))
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a == 10**18))
users_list[0]

Started for 16 users:
................

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '6a78cb4b4514091c91e81efcb73e132f0e050345',
 'filename': 'UTC--2022-10-11T13-21-16.771622196Z--6a78cb4b4514091c91e81efcb73e132f0e050345',
 'attach': '/tmp/attach/6a78cb4b4514091c91e81efcb73e132f0e050345.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xf85c1a0d3c5349d234b1392a32ea23b749c10d8f9161f8aca8fa8cf4dc6be10b',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xd1cEE4d9114ba7c29489bfd65487Dd04B39cd386',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x3a3aa71994d7f4f7fb95cf576d216b57a20e2d6f26077f848fb3c9dc6100e4c2',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0x5d37ef7d151A7c2bB7F491CaB359aB618c8a08B0',
 'badparity_lib': '0x856D84841B7d77e9f273CC5Fc4A55df10B1AF75a',
 'daodown_tx': '0x242de5bde50ebb3b1e4df1723b6f206296a2f7d8a7fe3ebfcc55fb7faf7eb48e',
 'daodo

In [111]:
# copy challenge metadata
challenge_path=CHALLENGES_PATH + "/daodown/publish/*"
attach_path=GEN_CONFIG_PATH + "/attach"
challenge_name="daodown"
copy_challenge_metadata(users_list,challenge_path,challenge_name,attach_path)

/smartenv/setup/generated_config//attach/df989609ed61dea253bb838927c66818142384a7/daodown
/smartenv/setup/generated_config//attach/ec783a2c453025391c4e70875c8cf4f34fa2ab1b/daodown
/smartenv/setup/generated_config//attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0/daodown
/smartenv/setup/generated_config//attach/dc870b8a86f3d2201fbdbad4e3995154c416e889/daodown
/smartenv/setup/generated_config//attach/6a78cb4b4514091c91e81efcb73e132f0e050345/daodown
/smartenv/setup/generated_config//attach/f15e7785122b2725c5438d79b4ea302c48961f28/daodown
/smartenv/setup/generated_config//attach/30b0fe2ed4e20e16944d9f032cfb0e9067eb5479/daodown
/smartenv/setup/generated_config//attach/0a25d85f850060cfcd6bbbf95825ed0e8ef24e6b/daodown
/smartenv/setup/generated_config//attach/b69743e47ec0dcbc45ea0260c96469d72e56fd0a/daodown
/smartenv/setup/generated_config//attach/c0af8d74d633cca61ddd57f122aef2d03ce20757/daodown
/smartenv/setup/generated_config//attach/5e5d6f287e50fd9308634c2ac892c6a6e02506c3/daodown
/smartenv/

## Challenge: Faildice

In [112]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '6a78cb4b4514091c91e81efcb73e132f0e050345',
 'filename': 'UTC--2022-10-11T13-21-16.771622196Z--6a78cb4b4514091c91e81efcb73e132f0e050345',
 'attach': '/tmp/attach/6a78cb4b4514091c91e81efcb73e132f0e050345.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xf85c1a0d3c5349d234b1392a32ea23b749c10d8f9161f8aca8fa8cf4dc6be10b',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xd1cEE4d9114ba7c29489bfd65487Dd04B39cd386',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x3a3aa71994d7f4f7fb95cf576d216b57a20e2d6f26077f848fb3c9dc6100e4c2',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0x5d37ef7d151A7c2bB7F491CaB359aB618c8a08B0',
 'badparity_lib': '0x856D84841B7d77e9f273CC5Fc4A55df10B1AF75a',
 'daodown_tx': '0x242de5bde50ebb3b1e4df1723b6f206296a2f7d8a7fe3ebfcc55fb7faf7eb48e',
 'daodo

In [113]:
!ls -la /usr/local/bin/solc*

-rwxr-xr-x 1 root root 11833912 Jun 15  2021 /usr/local/bin/solc
-rwxr-xr-x 1 root root  6284040 Feb 12  2019 /usr/local/bin/solc_5.4
-rwxr-xr-x 1 root root 11192752 Oct 19  2020 /usr/local/bin/solc_7.4
-rwxr-xr-x 1 root root 11833912 Apr 21  2021 /usr/local/bin/solc_8.4


In [114]:
!solc --version

solc, the solidity compiler commandline interface
Version: 0.8.4+commit.c7e474f2.Linux.g++


In [115]:
!solc --overwrite -o "{CHALLENGES_PATH}/faildice/publish/" --abi --bin "{CHALLENGES_PATH}/faildice/publish/SatoshiFailDice.sol"

Compiler run successful. Artifact(s) can be found in directory /smartenv/challenges//faildice/publish/.


In [116]:
path= CHALLENGES_PATH + "/faildice/publish/SatoshiFailDice.sol"
challange_name="faildice"
seedvalue=SEED_VALUE_FAILDICE

forall_compile_and_deploy(users_list,
                          path,
                          challange_name,
                          seedvalue,
                          argument2=True)
users_list[0]

Started for 16 users:
~~~~~~~~~~~~~~~~----------------

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '6a78cb4b4514091c91e81efcb73e132f0e050345',
 'filename': 'UTC--2022-10-11T13-21-16.771622196Z--6a78cb4b4514091c91e81efcb73e132f0e050345',
 'attach': '/tmp/attach/6a78cb4b4514091c91e81efcb73e132f0e050345.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xf85c1a0d3c5349d234b1392a32ea23b749c10d8f9161f8aca8fa8cf4dc6be10b',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xd1cEE4d9114ba7c29489bfd65487Dd04B39cd386',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x3a3aa71994d7f4f7fb95cf576d216b57a20e2d6f26077f848fb3c9dc6100e4c2',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0x5d37ef7d151A7c2bB7F491CaB359aB618c8a08B0',
 'badparity_lib': '0x856D84841B7d77e9f273CC5Fc4A55df10B1AF75a',
 'daodown_tx': '0x242de5bde50ebb3b1e4df1723b6f206296a2f7d8a7fe3ebfcc55fb7faf7eb48e',
 'daodo

In [117]:
path= CHALLENGES_PATH + "/faildice/publish/SatoshiFailDice.sol"
challange_name="faildice"
forall_check_contract(users_list,path,challange_name)
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a < b))
#forall_check_contract(users_list,path,challange_name,predicate=(lambda a, b: a == 10**18))
users_list[0]

Started for 16 users:
................

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '6a78cb4b4514091c91e81efcb73e132f0e050345',
 'filename': 'UTC--2022-10-11T13-21-16.771622196Z--6a78cb4b4514091c91e81efcb73e132f0e050345',
 'attach': '/tmp/attach/6a78cb4b4514091c91e81efcb73e132f0e050345.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xf85c1a0d3c5349d234b1392a32ea23b749c10d8f9161f8aca8fa8cf4dc6be10b',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xd1cEE4d9114ba7c29489bfd65487Dd04B39cd386',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x3a3aa71994d7f4f7fb95cf576d216b57a20e2d6f26077f848fb3c9dc6100e4c2',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0x5d37ef7d151A7c2bB7F491CaB359aB618c8a08B0',
 'badparity_lib': '0x856D84841B7d77e9f273CC5Fc4A55df10B1AF75a',
 'daodown_tx': '0x242de5bde50ebb3b1e4df1723b6f206296a2f7d8a7fe3ebfcc55fb7faf7eb48e',
 'daodo

In [118]:
# copy challenge metadata
challenge_path= CHALLENGES_PATH + "/faildice/publish/*"
attach_path=GEN_CONFIG_PATH + "/attach"
challenge_name="faildice"
copy_challenge_metadata(users_list,challenge_path,challenge_name,attach_path)

/smartenv/setup/generated_config//attach/df989609ed61dea253bb838927c66818142384a7/faildice
/smartenv/setup/generated_config//attach/ec783a2c453025391c4e70875c8cf4f34fa2ab1b/faildice
/smartenv/setup/generated_config//attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0/faildice
/smartenv/setup/generated_config//attach/dc870b8a86f3d2201fbdbad4e3995154c416e889/faildice
/smartenv/setup/generated_config//attach/6a78cb4b4514091c91e81efcb73e132f0e050345/faildice
/smartenv/setup/generated_config//attach/f15e7785122b2725c5438d79b4ea302c48961f28/faildice
/smartenv/setup/generated_config//attach/30b0fe2ed4e20e16944d9f032cfb0e9067eb5479/faildice
/smartenv/setup/generated_config//attach/0a25d85f850060cfcd6bbbf95825ed0e8ef24e6b/faildice
/smartenv/setup/generated_config//attach/b69743e47ec0dcbc45ea0260c96469d72e56fd0a/faildice
/smartenv/setup/generated_config//attach/c0af8d74d633cca61ddd57f122aef2d03ce20757/faildice
/smartenv/setup/generated_config//attach/5e5d6f287e50fd9308634c2ac892c6a6e02506c3/faildice

## Add optional/additional meta data

In [119]:
def add_login_token(path,users):
    login_tokens=list()
    with open(path,'r') as f_input:
        for line in f_input:
            #print(line.strip())
            login_tokens.append(line.strip())    
    
    for i,u in enumerate(users):
        u["login_token"] = login_tokens[i]
    
    return login_tokens.copy()

In [120]:
login_tokens = add_login_token(PARTICIPANTS_PATH + "login_tokens",users_list)

In [121]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '6a78cb4b4514091c91e81efcb73e132f0e050345',
 'filename': 'UTC--2022-10-11T13-21-16.771622196Z--6a78cb4b4514091c91e81efcb73e132f0e050345',
 'attach': '/tmp/attach/6a78cb4b4514091c91e81efcb73e132f0e050345.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xf85c1a0d3c5349d234b1392a32ea23b749c10d8f9161f8aca8fa8cf4dc6be10b',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xd1cEE4d9114ba7c29489bfd65487Dd04B39cd386',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x3a3aa71994d7f4f7fb95cf576d216b57a20e2d6f26077f848fb3c9dc6100e4c2',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0x5d37ef7d151A7c2bB7F491CaB359aB618c8a08B0',
 'badparity_lib': '0x856D84841B7d77e9f273CC5Fc4A55df10B1AF75a',
 'daodown_tx': '0x242de5bde50ebb3b1e4df1723b6f206296a2f7d8a7fe3ebfcc55fb7faf7eb48e',
 'daodo

In [122]:
# login_tokens

## Write Students

In [123]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '6a78cb4b4514091c91e81efcb73e132f0e050345',
 'filename': 'UTC--2022-10-11T13-21-16.771622196Z--6a78cb4b4514091c91e81efcb73e132f0e050345',
 'attach': '/tmp/attach/6a78cb4b4514091c91e81efcb73e132f0e050345.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xf85c1a0d3c5349d234b1392a32ea23b749c10d8f9161f8aca8fa8cf4dc6be10b',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xd1cEE4d9114ba7c29489bfd65487Dd04B39cd386',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x3a3aa71994d7f4f7fb95cf576d216b57a20e2d6f26077f848fb3c9dc6100e4c2',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0x5d37ef7d151A7c2bB7F491CaB359aB618c8a08B0',
 'badparity_lib': '0x856D84841B7d77e9f273CC5Fc4A55df10B1AF75a',
 'daodown_tx': '0x242de5bde50ebb3b1e4df1723b6f206296a2f7d8a7fe3ebfcc55fb7faf7eb48e',
 'daodo

In [124]:
import json

def persist_participants_json(path,users):
    with open(path,'w') as f_output:
        json.dump(users,f_output)

In [125]:
out_file_json = GEN_CONFIG_PATH + "/participants_meta.json"
persist_participants_json(out_file_json,users_list)
len(users_list)

16

In [126]:
import csv

def persist_participants_csv(path,
                   out_fieldnames=['firstname', 'lastname','ppid','email','cc','attach','contract','account'],
                   cc_mail="ajudmayer@sba-research.org"):
    with open(path,'w',encoding='utf-8') as f_output:
    #with open(path,'w',encoding='latin-1') as f_output:
        writer = csv.DictWriter(f_output, 
                                fieldnames=out_fieldnames, 
                                delimiter=',',
                                quotechar='\"',
                                extrasaction='ignore')
        writer.writeheader()
        for i,row in enumerate(users_list):
            row["cc"]=cc_mail
            #print(row)
            writer.writerow(row)
    return i+1

In [127]:
out_file = GEN_CONFIG_PATH + "/participants_meta.csv"
out_fieldnames = ['ppid','email','cc','attach','account']
#out_fieldnames = ['firstname', 'lastname','matr','email','cc','attach','contract','account']
#out_fieldnames = ['firstname', 
#                  'lastname',
#                  'matr',
#                  'email',
#                  'cc',
#                  'attach',
#                  'account',
#                  'notawallet_addr',
#                 ]
cc_mail = "ajudmayer@sba-research.org"
persist_participants_csv(out_file,out_fieldnames,cc_mail)

16

In [128]:
!head -n2 $out_file

In [129]:
!wc -l $out_file

17 /smartenv/setup/generated_config//participants_meta.csv


## Prepare mail

In [130]:
users_list[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '6a78cb4b4514091c91e81efcb73e132f0e050345',
 'filename': 'UTC--2022-10-11T13-21-16.771622196Z--6a78cb4b4514091c91e81efcb73e132f0e050345',
 'attach': '/tmp/attach/6a78cb4b4514091c91e81efcb73e132f0e050345.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0xf85c1a0d3c5349d234b1392a32ea23b749c10d8f9161f8aca8fa8cf4dc6be10b',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0xd1cEE4d9114ba7c29489bfd65487Dd04B39cd386',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0x3a3aa71994d7f4f7fb95cf576d216b57a20e2d6f26077f848fb3c9dc6100e4c2',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0x5d37ef7d151A7c2bB7F491CaB359aB618c8a08B0',
 'badparity_lib': '0x856D84841B7d77e9f273CC5Fc4A55df10B1AF75a',
 'daodown_tx': '0x242de5bde50ebb3b1e4df1723b6f206296a2f7d8a7fe3ebfcc55fb7faf7eb48e',
 'daodo

In [131]:
import os

def dump_addresses(users,attach_path=GEN_CONFIG_PATH + "/attach"):
    for u in users:
        arg_dest = attach_path + "/" + u["account"] + "/" + "addresses"
        ch_string = "notawallet_addr = " + u["notawallet_addr"]
        !echo $ch_string >> $arg_dest
        
        ch_string = "badparity_addr = " + u["badparity_addr"]
        !echo $ch_string >> $arg_dest    
        
        ch_string = "daodown_addr = " + u["daodown_addr"]
        !echo $ch_string >> $arg_dest
        
        ch_string = "faildice_addr = " + u["faildice_addr"]
        !echo $ch_string >> $arg_dest
        
        arg_dest = attach_path + "/" + u["account"] + "/" + "login_token"
        ch_string = "login_token = " + u["login_token"]
        !echo $ch_string >> $arg_dest
        
        print(".",end="")

In [132]:
# write all addresses to folders 
dump_addresses(users_list,attach_path=GEN_CONFIG_PATH + "/attach")

................

In [133]:
# copy general information i.e., datadir to individual folders
source_path=GLOBAL_INFO_PATH
attach_path=GEN_CONFIG_PATH + "/attach"
challenge_name=""
copy_challenge_metadata(users_list,source_path,challenge_name,attach_path)

/smartenv/setup/generated_config//attach/df989609ed61dea253bb838927c66818142384a7/
/smartenv/setup/generated_config//attach/ec783a2c453025391c4e70875c8cf4f34fa2ab1b/
/smartenv/setup/generated_config//attach/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0/
/smartenv/setup/generated_config//attach/dc870b8a86f3d2201fbdbad4e3995154c416e889/
/smartenv/setup/generated_config//attach/6a78cb4b4514091c91e81efcb73e132f0e050345/
/smartenv/setup/generated_config//attach/f15e7785122b2725c5438d79b4ea302c48961f28/
/smartenv/setup/generated_config//attach/30b0fe2ed4e20e16944d9f032cfb0e9067eb5479/
/smartenv/setup/generated_config//attach/0a25d85f850060cfcd6bbbf95825ed0e8ef24e6b/
/smartenv/setup/generated_config//attach/b69743e47ec0dcbc45ea0260c96469d72e56fd0a/
/smartenv/setup/generated_config//attach/c0af8d74d633cca61ddd57f122aef2d03ce20757/
/smartenv/setup/generated_config//attach/5e5d6f287e50fd9308634c2ac892c6a6e02506c3/
/smartenv/setup/generated_config//attach/3dae9b70b7e263c404e0ca2f3f68e9002ecccaf9/
/sma

In [134]:
# create archives
import os

def create_user_archives(users,attach_path=GEN_CONFIG_PATH + "/attach",attach_tgz=GEN_CONFIG_PATH + "/attach_tgz"):
    !mkdir -p $attach_tgz
    for d in os.listdir(attach_path):
        #print(d)
        if os.path.isdir(os.path.join(attach_path, d)):
            #print(d)
            arg_src_dir = attach_path + "/" + d 
            arg_dst_dir = attach_tgz + "/" + d + ".tar.gz"
            print(arg_dst_dir)
            #With verbose output:
            #!tar -cvzf $arg_dst_dir -C $arg_src_dir . 
            #Without verbose output
            !tar -czf $arg_dst_dir -C $arg_src_dir . 

In [135]:
attach_path=GEN_CONFIG_PATH + "/attach"
attach_tgz=GEN_CONFIG_PATH + "/attach_tgz"
create_user_archives(users_list,attach_path,attach_tgz)

/smartenv/setup/generated_config//attach_tgz/df989609ed61dea253bb838927c66818142384a7.tar.gz
/smartenv/setup/generated_config//attach_tgz/ec783a2c453025391c4e70875c8cf4f34fa2ab1b.tar.gz
/smartenv/setup/generated_config//attach_tgz/9ddbee8f1bb05273118d8e802797e5fb93fc1bc0.tar.gz
/smartenv/setup/generated_config//attach_tgz/dc870b8a86f3d2201fbdbad4e3995154c416e889.tar.gz
/smartenv/setup/generated_config//attach_tgz/6a78cb4b4514091c91e81efcb73e132f0e050345.tar.gz
/smartenv/setup/generated_config//attach_tgz/f15e7785122b2725c5438d79b4ea302c48961f28.tar.gz
/smartenv/setup/generated_config//attach_tgz/30b0fe2ed4e20e16944d9f032cfb0e9067eb5479.tar.gz
/smartenv/setup/generated_config//attach_tgz/0a25d85f850060cfcd6bbbf95825ed0e8ef24e6b.tar.gz
/smartenv/setup/generated_config//attach_tgz/b69743e47ec0dcbc45ea0260c96469d72e56fd0a.tar.gz
/smartenv/setup/generated_config//attach_tgz/c0af8d74d633cca61ddd57f122aef2d03ce20757.tar.gz
/smartenv/setup/generated_config//attach_tgz/5e5d6f287e50fd9308634c2ac

## Test loading a key

In [ ]:
#w3.eth.accounts

In [ ]:
#w3.geth.personal.unlockAccount(w3.eth.accounts[3],"INSERT",0)

In [ ]:
#w3.fromWei(w3.eth.getBalance(w3.eth.accounts[3]),'ether')

## Load Participants again and attach meta data

In [142]:
import json

def load_users(path):
    with open(path,'r') as f_input:
        return json.load(f_input)

In [143]:
users_loaded = load_users(GEN_CONFIG_PATH + "/participants_meta.json")
users_loaded[0]

{'firstname': 'Aljosha',
 'lastname': 'Judmayer',
 'email': 'ajudmayer@sba-research.org',
 'ppid': '1',
 'account': '032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'filename': 'UTC--2021-06-29T11-42-22.127396867Z--032fcb0a1c233da1546d2242ce0b4c4a51118ec3',
 'attach': '/tmp/attach/032fcb0a1c233da1546d2242ce0b4c4a51118ec3.tar.gz',
 'seedbalance': 16000000000000000000,
 'balance': 16000000000000000000,
 'notawallet_tx': '0x7f0cf6b9f07eaebd7ec993fb4491124b72a3d7a190996d4cda9bd32830a19fdb',
 'notawallet_seedvalue': 10000000000000000000,
 'notawallet_addr': '0x5341d88F2CC84e9Ca4F420Fd70627C7669570375',
 'notawallet_balance': 10000000000000000000,
 'badparity_tx': '0xa5ea7d38113b5adea43b1606a3ebadbcc941b8b097a8806a47ec913c12e262c5',
 'badparity_seedvalue': 30000000000000000000,
 'badparity_addr': '0xc7253Ea7f8b505C74dC9b205a9a17FeFb77Ba817',
 'badparity_lib': '0xfC97cd6Ac9edD2E5Db2B02Fa633456D79E2516A0',
 'daodown_tx': '0x9d78225045da97854620fc64f362ccd7d5b01a157368485649f6b8d7e0eb3858',
 'daodo